# dark-vessel-hunter
DTU Deep Learning project 29, group 80

### Run this in your terminal before executing this:
```bash
pip install -r requirements.txt
```

## Import of the files

In [1]:
import ais_downloader

/opt/anaconda3/envs/dl/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Data setup
### Set start and end date, and folder data name

In [2]:
START_DATE = "2025-02-14"
END_DATE   = "2025-02-14"

FOLDER_NAME = "ais-data"

### Imports

In [3]:
### Imports
from tqdm import tqdm
from pathlib import Path

### Script

In [4]:
### Script
# --- Create folder path ---
folder_path = Path(FOLDER_NAME)

# --- Build the schedule of download string dates ---
dates = ais_downloader.get_work_dates(START_DATE, END_DATE, folder_path)

# --- Iterate with tqdm and download, unzip and delete ---
for d in tqdm(dates, desc="Processing download, unzip and delete", unit="file"):
    ais_downloader.download_ais_data_one_day(d, folder_path)
    

aisdk-2025-02-14.zip: 100%|██████████| 500M/500M [00:19<00:00, 27.3MB/s]ile/s]
Processing download, unzip and delete: 100%|██████████| 1/1 [00:23<00:00, 23.90s/file]
